In [ ]:
import os

In [ ]:
directory = "results/classifications/20220304/"
suffix= '20220304'

## Mosiac tifs

In [ ]:
#build vrt
cwd = os.getcwd()
os.chdir(directory)
os.system("gdalbuildvrt " + "Indian_ocean_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=30 "\
       "--config GDAL_CACHEMAX 512 "
       + "Indian_ocean_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Indian_ocean_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

os.chdir(cwd)

## Clip mosaic to AEZ

In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [ ]:
create_local_dask_cluster()

In [ ]:
aez_region = 'data/Central.shp'
pred_tif = "results/classifications/"+suffix+"/Indian_ocean_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':5000, 'y':5000}

In [ ]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [ ]:
aez = xr_rasterize(aez, da)

In [ ]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [ ]:
%%time
da = da.compute()

In [ ]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True)